In [5]:
#import json
import pandas as pd
#import numpy as np
import sys
import os
import shutil
sys.path.insert(0,'/Users/sanjivjha/IMP-OIC/')
from utils.extractframes import FrameExtractor
import graphene
from STAR_eval.gpt_ask import run_gpt
directory_path = '/Users/sanjivjha/Library/CloudStorage/OneDrive-UniversitätSt.Gallen/Charades'#'STAR_eval/Charades'


In [6]:
QA_ds = pd.read_json('STAR_eval/STAR_val.json')
#QA_ds.head()

In [8]:
QA_ds_questions = QA_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]

In [9]:
QA = QA_ds_questions.groupby(['question_id','video_id']).sum().reset_index()#.to_csv('STAR_eval/question_set.csv')
#QA.head()

In [11]:
CLASSES = ['airplane', 'animal', 'arm', 'bag', 'banana', 'basket', 'beach', 'bear', 'bed', 'bench', 'bike',
                'bird', 'board', 'boat', 'book', 'boot', 'bottle', 'bowl', 'box', 'boy', 'branch', 'building',
                'bus', 'cabinet', 'cap', 'car', 'cat', 'chair', 'child', 'clock', 'coat', 'counter', 'cow', 'cup',
                'curtain', 'desk', 'dog', 'door', 'drawer', 'ear', 'elephant', 'engine', 'eye', 'face', 'fence',
                'finger', 'flag', 'flower', 'food', 'fork', 'fruit', 'giraffe', 'girl', 'glass', 'glove', 'guy',
                'hair', 'hand', 'handle', 'hat', 'head', 'helmet', 'hill', 'horse', 'house', 'jacket', 'jean',
                'kid', 'kite', 'lady', 'lamp', 'laptop', 'leaf', 'leg', 'letter', 'light', 'logo', 'man', 'men',
                'motorcycle', 'mountain', 'mouth', 'neck', 'nose', 'number', 'orange', 'pant', 'paper', 'paw',
                'people', 'person', 'phone', 'pillow', 'pizza', 'plane', 'plant', 'plate', 'player', 'pole', 'post',
                'pot', 'racket', 'railing', 'rock', 'roof', 'room', 'screen', 'seat', 'sheep', 'shelf', 'shirt',
                'shoe', 'short', 'sidewalk', 'sign', 'sink', 'skateboard', 'ski', 'skier', 'sneaker', 'snow',
                'sock', 'stand', 'street', 'surfboard', 'table', 'tail', 'tie', 'tile', 'tire', 'toilet', 'towel',
                'tower', 'track', 'train', 'tree', 'truck', 'trunk', 'umbrella', 'vase', 'vegetable', 'vehicle',
                'wave', 'wheel', 'window', 'windshield', 'wing', 'wire', 'woman', 'zebra']

REL_CLASSES = ['above', 'across', 'against', 'along', 'and', 'at', 'attached to', 'behind',
                'belonging to', 'between', 'carrying', 'covered in', 'covering', 'eating', 'flying in', 'for',
                'from', 'growing on', 'hanging from', 'has', 'holding', 'in', 'in front of', 'laying on',
                'looking at', 'lying on', 'made of', 'mounted on', 'near', 'of', 'on', 'on back of', 'over',
                'painted on', 'parked on', 'part of', 'playing', 'riding', 'says', 'sitting on', 'standing on',
                'to', 'under', 'using', 'walking in', 'walking on', 'watching', 'wearing', 'wears', 'with']

words = set(CLASSES+REL_CLASSES)


In [12]:
import string
index_list = []
for q in QA.loc[:,'question_id'].values:
    questions = QA.query("question_id=='"+q+"'")['question']
    choices = QA.query("question_id=='"+q+"'")['choices']
    list_choices = ''
    for ch in choices:
        for c in ch:
            list_choices+=c['choice'].translate(str.maketrans('', '', string.punctuation))
    count = 0
    question = ''
    for que in questions:
        question += que.translate(str.maketrans('', '', string.punctuation))
    
    for i in words:
        if i in (list((question.lower()).split(" "))+list((list_choices.lower()).split(" "))):
            if i not in  ['a', 'is', 'man', 'woman', 'boy', 'girl', 'person','people','scene','frame','in','to','the', 'of', 'on','with', 'from', 'at', 'and']:
                count+=1
    if count == 0:
        index_list.append(QA[QA['question_id']==q].index[0])

print(index_list)  
QA.drop(index_list, inplace = True)      
QA.info()

[0, 2, 3, 4, 9, 11, 12, 13, 15, 16, 17, 20, 24, 26, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50, 52, 55, 57, 58, 60, 61, 62, 63, 70, 72, 77, 78, 80, 82, 83, 84, 86, 87, 89, 90, 91, 92, 93, 94, 124, 127, 135, 136, 150, 156, 159, 164, 166, 172, 173, 174, 175, 183, 184, 189, 190, 197, 198, 199, 201, 204, 205, 211, 217, 221, 223, 224, 227, 229, 232, 234, 247, 256, 265, 266, 273, 282, 283, 284, 292, 293, 296, 298, 306, 307, 314, 316, 317, 321, 327, 328, 329, 344, 353, 354, 365, 368, 371, 378, 380, 381, 387, 390, 391, 394, 401, 404, 408, 413, 414, 425, 428, 429, 435, 436, 437, 441, 444, 447, 451, 464, 468, 469, 481, 483, 488, 489, 490, 491, 492, 494, 499, 501, 502, 503, 504, 506, 508, 510, 513, 514, 515, 517, 518, 520, 521, 522, 523, 524, 526, 528, 530, 533, 534, 535, 539, 544, 546, 547, 548, 549, 550, 552, 553, 554, 555, 560, 565, 566, 568, 569, 572, 574, 580, 583, 585, 586, 587, 588, 590, 592, 594, 596, 597, 598, 599, 604, 606, 607, 608, 609, 610, 612, 613, 614, 615, 616, 617, 618, 619,

In [18]:
QA.to_csv('STAR_eval/question_set.csv')

In [22]:
import ffmpeg

def segment_video(input_path, output_path, start, end):
    input_file = ffmpeg.input(directory_path+'/'+input_path+'.mp4')
    duration = int(end-start)
    output_file = ffmpeg.output(input_file.trim(start=start, duration = duration).filter('setpts','PTS-STARTPTS'), directory_path+'/trim/'+output_path+'.mp4')
    ffmpeg.run(output_file)

In [20]:
def run_oic(question_id, video_id, question, choices, start,end):
    TEMP_DIR = "temp"
    CAM_PATH = "cam"
    OUT_DIR = "out"

    if os.path.isdir(TEMP_DIR):
        shutil.rmtree(TEMP_DIR) 
    
    for video in os.scandir(directory_path):
        if video.is_file():
            if os.path.splitext(video.name)[0] == video_id:
                print('processing {}'.format(video_id))
                print(video_id, question, choices, start, end)
                segment_video(os.path.splitext(video.name)[0],question_id+os.path.splitext(video.name)[0],start,end)
                
                ex = FrameExtractor(30, directory_path+'/trim/'+question_id+video.name)
                ex.main()
                g = graphene.Graphene(0.3, 0.6)

                if not os.path.isdir(OUT_DIR):
                    os.mkdir(OUT_DIR)
                text = question_id+os.path.splitext(video.name)[0]+'graph2text.txt'
                img_path = directory_path+'/trim/'+question_id+os.path.splitext(video.name)[0]+'-opencv'
                #visual = question_id+os.path.splitext(video.name)[0]+'tg.png'
                #g.run_online(text)

                g.classify_images(img_path)
                graph_path = TEMP_DIR
                g.generate_temporal_graph_frames(graph_path, img_path)

                #g.generate_temporal_graph(graph_path)

                #g.tg.to_plot(os.path.join(OUT_DIR, visual))

                g.tg.to_text(os.path.join(OUT_DIR,text))
                
                if os.path.isfile(OUT_DIR+'/'+text):
                    with open(OUT_DIR+'/'+text) as f:
                        context = f.readline()
                        print(context)
                    
                        return run_gpt(context, question + choices)

In [24]:
q_ids = QA['question_id'].unique()

for q in q_ids:
    choices = 'Options: '
    que = QA.query("question_id == '"+q+"'")
    video_id = que['video_id'].values[0]
    question = que['question'].values[0]
    start = que['start'].values[0]
    end = que['end'].values[0]
    for choice in que['choices']:
        for c in choice:
            choices+=c['choice']
    
    answer = run_oic(q, video_id, question, choices, start, end)
    QA.loc[QA['question_id'] == q, 'OIC_answers'] = str(answer)
    print(answer)
    
QA.to_csv('STAR_eval/question_answers.csv')
    
    

processing SMVKB
SMVKB What else is the person able to do with the door? Options: Close the door.Wash the door.Open the door.Take the door. 16.5 23.6


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disab

KeyboardInterrupt: 